In [ ]:
import os
import textwrap
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import imageio
from moviepy import VideoFileClip, AudioFileClip  # ✅ new import style for MoviePy v2+

# === CONFIG ===
comic_panels = [
    {"image": "happy.jpg", "text": "I can't believe another year has flown by!"},
    {"image": "pensive.jpg", "text": "My brain's been running on so many tabs I forgot to clear the cache."},
    {"image": "sad.jpg", "text": "I wish I could go to beach, it's been long. I need some vitamin sea!"},
    {"image": "determined.jpg", "text": "But hey, it's a new year! Time to optimize my life and conquer some new horizons!"},
    {"image": "pointing.jpg", "text": "Firstly I need to find perfectly delicious meal I can possibly eat today."},
    {"image": "thinking.jpg", "text": "Maybe something with fresh seafood... right on the sand... I can almost smell the salt air."},
    {"image": "excited.jpg", "text": "Speaking of smells... wait, is that the scent of freshly baked birthday carbs?"},
    {"image": "me.jpg", "text": "Don't get too distracted by food! I've been running your life's algorithms all day..."},
    {"image": "noise.jpg", "text": "Wait… I heard a sound, was that a creak? Or just my imagination?"},
    {"image": "investigating.jpg", "text": "Wait... what's that behind the couch?"},
    {"image": "gift.jpg", "text": "A present! You're the best! "},
    {"image": "ending.jpg", "text": "Let's plan a trip soon..!!"}
]

video_file = "comic_video.mp4"
final_video = "comic_with_music.mp4"
audio_file = "background.mp3"  # 🎵 your downloaded background music
font_path = "comic.ttf"
bubble_scale = 0.42
line_spacing = 16
fps = 10
display_seconds = 4
fade_frames = 10


# === DRAW SPEECH BUBBLE ===
def draw_comic_bubble(draw, x, y, w, h, outline_color=(0, 0, 0, 255), fill_color=(255, 255, 255, 200)):
    radius = h // 4
    draw.rounded_rectangle([x, y, x + w, y + h],
                           radius=radius, fill=fill_color,
                           outline=outline_color, width=2)


# === PANEL RENDERING ===
def draw_panel_with_bubble(panel):
    img = Image.open(panel["image"]).convert("RGBA")
    panel_w, panel_h = img.size
    draw = ImageDraw.Draw(img, 'RGBA')

    try:
        font = ImageFont.truetype(font_path, 32)
    except:
        font = ImageFont.truetype("arial.ttf", 32)

    bubble_w = int(panel_w * bubble_scale)
    bubble_h = int(panel_h * bubble_scale * 0.5)
    bubble_y = 20
    bubble_x = (panel_w - bubble_w)//2

    draw_comic_bubble(draw, bubble_x, bubble_y, bubble_w, bubble_h)

    chars_per_line = max(10, bubble_w // 18)
    wrapped_text = textwrap.fill(panel["text"], width=chars_per_line)
    text_lines = wrapped_text.split("\n")
    line_height = font.getbbox("A")[3] - font.getbbox("A")[1]
    text_block_height = (line_height + line_spacing) * len(text_lines) - line_spacing
    text_y = bubble_y + (bubble_h - text_block_height) // 2

    for line in text_lines:
        text_w = draw.textlength(line, font=font)
        text_x = bubble_x + (bubble_w - text_w) // 2
        draw.text((text_x, text_y), line, font=font, fill=(0, 0, 0, 255))
        text_y += line_height + line_spacing

    return img.convert("RGB")


# === CREATE VIDEO ===
def create_video_from_panels(panels, video_output, fps=10, display_seconds=4, fade_frames=10):
    frames = []
    images = [draw_panel_with_bubble(panel) for panel in panels]
    display_frames = display_seconds * fps

    for i, img in enumerate(images):
        frames.extend([np.array(img)] * display_frames)
        if i < len(images) - 1:
            next_img = np.array(images[i+1])
            for f in range(1, fade_frames + 1):
                alpha = f / (fade_frames + 1)
                blended = np.array(img)*(1-alpha) + next_img*alpha
                frames.append(blended.astype(np.uint8))

    imageio.mimwrite(video_output, frames, fps=fps)
    print(f"✅ Video created successfully: {video_output}")


# === MERGE VIDEO + BACKGROUND MUSIC ===
def merge_video_audio(video_path, audio_path, output_path):
    if not os.path.exists(audio_path):
        print("⚠️ No background.mp3 found in the current folder. Skipping music merge.")
        os.rename(video_path, output_path)
        return

    print("🎬 Adding background music...")
    video = VideoFileClip(video_path)
    audio = AudioFileClip(audio_path)

    # Trim or loop audio to match video duration (v2+ safe way)
    if audio.duration > video.duration:
        audio = audio.with_duration(video.duration)
    else:
        audio = audio.loop(duration=video.duration)

    video = video.with_audio(audio)
    video.write_videofile(output_path, codec="libx264", audio_codec="aac")
    print(f"🎵 Final video with background music saved as: {output_path}")


# === RUN ===
if __name__ == "__main__":
    create_video_from_panels(comic_panels, video_file,
                             fps=fps, display_seconds=display_seconds, fade_frames=fade_frames)
    merge_video_audio(video_file, audio_file, final_video)


✅ Video created successfully: comic_video.mp4
🎬 Adding background music...
MoviePy - Building video comic_with_music.mp4.
MoviePy - Writing audio in comic_with_musicTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video comic_with_music.mp4



MoviePy - Done !
MoviePy - video ready comic_with_music.mp4
🎵 Final video with background music saved as: comic_with_music.mp4
